Prepare maps for analysis
- convert to z map
- check MNI space

Hypotheses:

Parametric effect of gain:

1. Positive effect in ventromedial PFC - for the equal indifference group
2. Positive effect in ventromedial PFC - for the equal range group
3. Positive effect in ventral striatum - for the equal indifference group
4. Positive effect in ventral striatum - for the equal range group

Parametric effect of loss:
- 5: Negative effect in VMPFC - for the equal indifference group
- 6: Negative effect in VMPFC - for the equal range group
- 7: Positive effect in amygdala - for the equal indifference group
- 8: Positive effect in amygdala - for the equal range group

Equal range vs. equal indifference:

- 9: Greater positive response to losses in amygdala for equal range condition vs. equal indifference condition.


In [39]:
import numpy,pandas
import nibabel
import json
import os,glob
import nilearn.image
import nilearn.input_data
import nilearn.plotting
from collections import OrderedDict
import shutil
import warnings
import sklearn
import matplotlib.pyplot as plt
import seaborn

%matplotlib inline

hypotheses= {1:'+gain: equal indiff',
            2:'+gain: equal range',
            3:'+gain: equal indiff',
            4:'+gain: equal range',
            5:'-loss: equal indiff',
            6:'-loss: equal range',
            7:'+loss: equal indiff',
            8:'+loss: equal range',
            9:'+loss:ER>EI'}


In [2]:
from templateflow.api import get
get('MNI152NLin2009cAsym', **{'from': 'MNI152NLin6Asym'})


Failed to import duecredit due to No module named 'duecredit'


PosixPath('/Users/poldrack/.cache/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5')

### loading the metadata


In [2]:
metadata_file = '/Users/poldrack/data_unsynced/NARPS/analysis_pipelines.xlsx'
metadata = pandas.read_excel(metadata_file,header=1)
metadata.shape

(70, 39)

In [3]:
metadata.columns

Index(['teamID', 'NV_collection_string', 'results_comments', 'preregistered',
       'link_preregistration_form', 'regions_definition', 'softwares',
       'n_participants', 'exclusions_details', 'used_fmriprep_data',
       'preprocessing_order', 'brain_extraction', 'segmentation',
       'slice_time_correction', 'motion_correction',
       'gradient_distortion_correction', 'intra_subject_coreg',
       'distortion_correction', 'inter_subject_reg', 'intensity_correction',
       'intensity_normalization', 'noise_removal', 'volume_censoring',
       'spatial_smoothing', 'preprocessing_comments',
       'data_submitted_to_model', 'spatial_region_modeled',
       'independent_vars_first_level', 'independent_vars_higher_level',
       'model_type', 'model_settings', 'inference_contrast_effect',
       'search_region', 'statistic_type', 'pval_computation',
       'multiple_testing_correction', 'comments_analysis', 'general comments',
       'additional files?'],
      dtype='object')

### Processing the statistical images

In [40]:
orig_dir = '/Users/poldrack/data_unsynced/NARPS/maps/orig'
output_dir = '/Users/poldrack/data_unsynced/NARPS/maps'

input_jsons = glob.glob(os.path.join(orig_dir,'*/images.json'))
print('found',len(input_jsons),'input directories')
input_dirs = [os.path.dirname(i) for i in input_jsons]
mask_img = os.path.join(output_dir,'templates/MNI152_T1_2mm_brain_mask.nii.gz')


found 57 input directories


In [6]:
# find maps
imgfile_dict = {'thresh':{},'unthresh':{}}

for i in input_dirs:
    for h in range(1,10):
        for t in ['thresh','unthresh']:
            fname=os.path.join(i,'hypo%d_%s.nii.gz'%(h,t))
            if os.path.exists(fname):
                if not i in imgfile_dict[t]:
                    imgfile_dict[t][i]={}
                imgfile_dict[t][i][h]=fname
    if i in imgfile_dict['thresh']:
        if not len(imgfile_dict['thresh'][i])==9:
            print("missing files for",i,': only',len(imgfile_dict['thresh'][i]),'found')
            del imgfile_dict['thresh'][i]
    if i in imgfile_dict['unthresh']:
        if not len(imgfile_dict['unthresh'][i])==9:
            print("missing files for",i,': only',len(imgfile_dict['unthresh'][i]),'found')
            del imgfile_dict['unthresh'][i]
for t in ['thresh','unthresh']:
    print(t,': found ',len(imgfile_dict[t]),'datasets')

thresh : found  60 datasets
unthresh : found  60 datasets


#### Resample into FSL MNI space

In [8]:
template_img = os.path.join(output_dir,'templates/MNI152_T1_2mm.nii.gz')

missing_images = []

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for i in input_dirs:
        if not (i in imgfile_dict['thresh']) or not (i in imgfile_dict['unthresh']):
            missing_images.append(i)
            continue
        if not (len(imgfile_dict['thresh'][i])==9) or not (len(imgfile_dict['unthresh'][i])==9):
            missing_images.append(i)
            continue
        # convert all to FSL space
        if 0:
            print(os.path.basename(i),': copying fmriprep data directly')
            target_dir=os.path.join(output_dir,'mnispace',os.path.basename(i))
            if not os.path.exists(target_dir):
                print('copying',i,'to',target_dir)
                shutil.copytree(i,target_dir)
        else:
            print(os.path.basename(i),':converting to FSL MNI space')
            outdir = i.replace('orig','resampled')
            if not os.path.exists(outdir):
                os.mkdir(outdir)
            for img in glob.glob(os.path.join(i,'hyp*nii.gz')):
                if os.path.basename(img).find('_thresh')>-1:
                    interp='nearest'
                else:
                    interp='continuous'
                resampled = nilearn.image.resample_to_img(img, template_img,
                                                    interpolation=interp)
                resampled.to_filename(os.path.join(outdir,os.path.basename(img)))

print('missing proper images for:')
[os.path.basename(i) for i in missing_images]

YZFBWTVU_Q6O0 :converting to FSL MNI space
FKMCNBTL_B5I6 :converting to FSL MNI space
CLTGPQPO_43FJ :converting to FSL MNI space
MQKTVNIH_08MQ :converting to FSL MNI space
RIIVGRDK_E3B6 :converting to FSL MNI space
YGVUUZDV_IZ20 :converting to FSL MNI space
JAWCZRDS_V55J :converting to FSL MNI space
MNASBORM_3PQ2 :converting to FSL MNI space
NPZYAJUF_94GU :converting to FSL MNI space
IZREEBAR_T54A :converting to FSL MNI space
CMGDLXNE_C22U :converting to FSL MNI space
COXUJKNY_R9K3 :converting to FSL MNI space
ZNLBVOZG_UK24 :converting to FSL MNI space
PVRJCPGN_X19V :converting to FSL MNI space
XTQTYBTF_0I4U :converting to FSL MNI space
QDWZHJWT_L7J7 :converting to FSL MNI space
PJLEEUYF_46CD :converting to FSL MNI space
CQCJIRXE_O03M :converting to FSL MNI space
WLJQPTCD_O6R6 :converting to FSL MNI space
LASLHJJB_I52Y :converting to FSL MNI space
WRZSQULI_B23O :converting to FSL MNI space
FLSSDFOD_9T8E :converting to FSL MNI space
QJVDBBPV_R42Q :converting to FSL MNI space
YWGVKXBZ_6F

[]

#### Combine thresholded images and create summary map

In [41]:
# make full image mask
full_mask_img = os.path.join(output_dir,'MNI152_all_voxels.nii.gz')

if not os.path.exists(full_mask_img):
    mi = nibabel.load(mask_img)
    d = mi.get_data()
    d = numpy.ones(d.shape)
    full_mask = nibabel.Nifti1Image(d,affine = mi.affine)
    full_mask.to_filename(full_mask_img)

for hyp in range(1,10):
    hmaps = glob.glob(os.path.join(output_dir,'resampled/*/hypo%d_thresh.nii.gz'%hyp))
    print('hypo %d: found %d maps'%(hyp,len(hmaps)))
    masker = nilearn.input_data.NiftiMasker(mask_img=full_mask_img)
    concat_data = masker.fit_transform(hmaps)
    concat_img = masker.inverse_transform(concat_data)
    concat_img.to_filename(os.path.join(output_dir,'thresh_concat/hypo%d.nii.gz'%hyp))


/Users/poldrack/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  


hypo 1: found 57 maps
hypo 2: found 57 maps
hypo 3: found 57 maps
hypo 4: found 57 maps
hypo 5: found 57 maps
hypo 6: found 57 maps
hypo 7: found 57 maps
hypo 8: found 57 maps
hypo 9: found 57 maps


In [37]:
# create overlap maps for thresholded maps

for hyp in range(1,10):
    concat_img=nibabel.load(os.path.join(output_dir,'thresh_concat/hypo%d.nii.gz'%hyp))
    concat_data = concat_img.get_data()
    concat_data = (concat_data>10e-6).astype('float')
    concat_data = numpy.nan_to_num(concat_data)
    concat_mean = numpy.mean(concat_data,3)
    concat_mean_img = nibabel.Nifti1Image(concat_mean,affine=concat_img.header.get_best_affine())
    concat_mean_img.to_filename(os.path.join(output_dir,'thresh_mean/hypo%d.nii.gz'%hyp))

#### Convert all unthresholded images to Z stat images

In [24]:
# load image info from json

def get_map_types(j):
    maptypes=[]
    for img in j:
        maptypes.append(img['map_type'])
    return(list(set(maptypes)))

def parse_json(j):
    imgtype = {}
    for img in j:
        fname=os.path.basename(img['file']).replace('.nii.gz','')
        print(fname)
        f_s = fname.split('_')
        try:
            hypnum = int(f_s[0].replace('hypo',''))
            imgtype = f_s[1]
        except:
            print("something went wrong")
        print(f_s)
    return(imgtype)
        

jsonfiles = glob.glob(os.path.join(orig_dir,'*/images.json'))
print("found %d json files"%len(jsonfiles))

for j in jsonfiles:
    with open(j) as f:
        info = json.load(f)
        print(get_map_types(info))

found 57 json files
['T map']
['T map']
['parcellation', 'other']
['T map']
['ROI/mask', 'univariate-beta map', 'T map']
['ROI/mask', 'T map']
['T map']
['other']
['T map']
['Z map', '1-P map ("inverted" probability)', 'other', 'univariate-beta map']
['other']
['T map']
['parcellation', 'other', 'ROI/mask']
['other']
['parcellation', 'other']
['other']
['T map']
['parcellation', 'other']
['other']
['other']
['T map']
['T map']
['Z map']
['Z map']
['other', 'T map']
['Z map']
['parcellation', 'other']
['Z map']
['other']
['Z map']
['Z map', 'other', 'T map']
['Z map']
['parcellation', 'other']
['parcellation', 'other']
['other']
['Z map']
['parcellation', 'other']
['T map']
['other']
['T map']
['parcellation', 'other', 'ROI/mask']
['T map']
['parcellation', 'other']
['parcellation', 'other']
['T map']
['other']
['T map']
['parcellation', 'other']
['Z map']
['other']
['Z map', 'other', 'univariate-beta map']
['other']
['other']
['parcellation', 'other']
['univariate-beta map', 'T map']
[

In [42]:
unthresh_hmaps={}
for hyp in range(1,10):
    hmaps = glob.glob(os.path.join(output_dir,'resampled/*/hypo%d_unthresh.nii.gz'%hyp))
    hmaps.sort()
    unthresh_hmaps[hyp]=hmaps
    print('hypo %d: found %d maps'%(hyp,len(hmaps)))
    masker = nilearn.input_data.NiftiMasker(mask_img=full_mask_img)
    concat_data = masker.fit_transform(hmaps)
    concat_img = masker.inverse_transform(concat_data)
    concat_img.to_filename(os.path.join(output_dir,'unthresh_concat/hypo%d.nii.gz'%hyp))
    # compute range
    concat_data=numpy.nan_to_num(concat_img.get_data())
    datarange = numpy.max(concat_data,axis=3) - numpy.min(concat_data,axis=3)
    range_img = nibabel.Nifti1Image(datarange,affine=concat_img.header.get_best_affine())
    range_img.to_filename(os.path.join(output_dir,'unthresh_range/hypo%d.nii.gz'%hyp))
    datavar = numpy.std(concat_data,axis=3)
    var_img = nibabel.Nifti1Image(datavar,affine=concat_img.header.get_best_affine())
    var_img.to_filename(os.path.join(output_dir,'unthresh_std/hypo%d.nii.gz'%hyp))
    
    

hypo 1: found 57 maps
hypo 2: found 57 maps
hypo 3: found 57 maps
hypo 4: found 57 maps
hypo 5: found 57 maps
hypo 6: found 57 maps
hypo 7: found 57 maps
hypo 8: found 57 maps
hypo 9: found 57 maps


In [26]:
hmaps[54]

'/Users/poldrack/data_unsynced/NARPS/maps/resampled/ZFFBNRMO_51PW/hypo9_unthresh.nii.gz'